# Web scraper for Yahoo finance

This is an R script, which uses RSelenium to scrape historical prices for a given security from Yahoo. It then writes them into a table and exports to an xlsx-file.

## Setting macrovariables

You can change the values for the variables below to fit your needs.

This variable is a list of all the tickers for the securities you want to scrape the historical prices for.

In [1]:
symbols = c("NFLX", "SPOT", "TLT")

The startDate-variable sets start date for the historical prices.

The endDate-variable sets the end date for the historical prices. To use the current day, set it equal to Sys.date().

In [2]:
startDate = "2021-09-01"
endDate = Sys.Date()

The path-variable sets the path for the xlsx-file for the table with all the scraped content.

In [3]:
path = "C:/Users/joona/Desktop/stocks.xlsx"

## Importing libraries and setting up remote driver

Before we start scraping, we need to import (and install if not installed) the needed libraries.

In [4]:
library(rvest)
library(dplyr)
library(RSelenium)
library(XML)
library(tidyverse)
library(lubridate)
library(zoo)
library(xlsx)

Loading required package: xml2
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
Registered S3 methods overwritten by 'tibble':
  method     from  
  format.tbl pillar
  print.tbl  pillar

Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Warning message:
"package 'RSelenium' was built under R version 3.6.3"Registered S3 method overwritten by 'openssl':
  method      from
  print.bytes Rcpp
Warning message:
"package 'XML' was built under R version 3.6.3"
Attaching package: 'XML'

The following object is masked from 'package:rvest':

    xml

-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
v ggplot2 3.3.5     v readr   1.3.1
v tibble  2.1.1     v purrr   0.3.2
v tidyr   0.8.3     v stringr 1.4.0
v ggplot2 3.3.5     v forcats 0.4.0
-- Conflicts ----------------------

If you do not have Docker set up already, install it before running the following script:

In [5]:
shell('docker pull selenium/standalone-chrome')
shell('docker run -d -p 4445:4444 --shm-size="2g" selenium/standalone-chrome')

Warning message in shell("docker run -d -p 4445:4444 --shm-size=\"2g\" selenium/standalone-chrome"):
"'docker run -d -p 4445:4444 --shm-size="2g" selenium/standalone-chrome' execution failed with error code 125"

Now let us assign a remote driver into a variable and connect to the remote driver:

In [6]:
remDr <- remoteDriver(remoteServerAddr = "localhost",
                     port = 4445L,
                     browserName = "chrome")
remDr$open()

[1] "Connecting to remote server"
$acceptInsecureCerts
[1] FALSE

$browserName
[1] "chrome"

$browserVersion
[1] "95.0.4638.54"

$chrome
$chrome$chromedriverVersion
[1] "95.0.4638.17 (a9d0719444d4b035e284ed1fce73bf6ccd789df2-refs/branch-heads/4638@{#178})"

$chrome$userDataDir
[1] "/tmp/.com.google.Chrome.lTvvF8"


$`goog:chromeOptions`
$`goog:chromeOptions`$debuggerAddress
[1] "localhost:35089"


$networkConnectionEnabled
[1] FALSE

$pageLoadStrategy
[1] "normal"

$platformName
[1] "linux"

$proxy
named list()

$`se:cdp`
[1] "ws://172.17.0.2:4444/session/de94179dba16f3ffe28dd88eddd76c96/se/cdp"

$`se:cdpVersion`
[1] "95.0.4638.54"

$`se:vnc`
[1] "ws://172.17.0.2:4444/session/de94179dba16f3ffe28dd88eddd76c96/se/vnc"

$`se:vncEnabled`
[1] TRUE

$`se:vncLocalAddress`
[1] "ws://172.17.0.2:7900"

$setWindowRect
[1] TRUE

$strictFileInteractability
[1] FALSE

$timeouts
$timeouts$implicit
[1] 0

$timeouts$pageLoad
[1] 300000

$timeouts$script
[1] 30000


$unhandledPromptBehavior
[1] "dismiss

## Scraping the data

Now we can do the actual scraping with the following code (this might take some time):

In [7]:
period1 = as.integer(difftime(time2 = "1970-01-01", time1 = startDate, units = "secs"))
period2 = as.integer(difftime(time2 = "1970-01-01", time1 =  endDate, units = "secs"))

table = list()

for (symbol in symbols) {
  # creates the URL
  link = paste0("https://finance.yahoo.com/quote/", symbol, "/history?period1=",
                   period1, "&period2=", period2,
                   "&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true&guccounter=2")
  
  # navigates to the link and scrolls down to load the whole table
  remDr$navigate(link)
  webElem <- remDr$findElement("css", "body")
  
  for (j in 1:6) {
    j = j+1
    Sys.sleep(3)
    webElem$sendKeysToElement(list(key = "end"))
  }

  # reads the table and assigns it to a variable
  doc <- htmlParse(remDr$getPageSource()[[1]])
  temptable = assign(paste0("table", symbol), readHTMLTable(doc, as.data.frame = TRUE)[[1]])
  
  #print final message
  if (symbol == symbols[length(symbols)]) {
    print("Scraping done!")
  }
  table[[symbol]] = temptable
}

[1] "Scraping done!"


And now we can left join all of the tables from the different securities.

Note! Left joining the tables means in this case, that the dates we got from scraping the data for the first security (i.e. the first item in the symbols-list) are used to join the data from the different tables. If we wanted to scrape data for securities in different markets, some important data might be lost.

In [8]:
final_table = table %>%
  reduce(left_join, by = "Date")
head(final_table)

Date,Open.x,High.x,Low.x,Close*.x,Adj Close**.x,Volume.x,Open.y,High.y,Low.y,Close*.y,Adj Close**.y,Volume.y,Open,High,Low,Close*,Adj Close**,Volume
"Oct 28, 2021",670.95,676.80,668.03,674.05,674.05,"2,798,729",273.62,293.32,273.14,288.95,288.95,"3,374,820",147.79,148.33,146.84,147.24,147.24,"17,496,320"
"Oct 27, 2021",669.00,671.41,661.85,662.92,662.92,"2,276,900",269.91,278.00,260.53,273.13,273.13,"3,958,200",146.40,148.18,145.93,147.74,147.74,"22,092,300"
"Oct 26, 2021",673.76,676.49,662.77,668.52,668.52,"2,904,800",260.34,263.69,251.57,252.20,252.20,"1,672,600",144.62,145.10,143.74,145.10,145.10,"13,671,000"
"Oct 25, 2021",663.74,675.88,657.07,671.66,671.66,"3,833,500",252.06,259.19,250.58,257.89,257.89,"1,009,600",143.67,144.30,143.63,143.91,143.91,"8,137,900"
"Oct 22, 2021",651.81,665.46,651.81,664.78,664.78,"6,179,700",251.50,254.37,249.02,252.96,252.96,"988,000",143.45,144.40,143.21,144.13,144.13,"13,738,700"
"Oct 21, 2021",628.89,654.01,628.65,653.16,653.16,"8,437,100",251.09,263.04,250.84,255.00,255.00,"1,093,500",143.00,143.15,142.23,142.56,142.56,"13,660,000"


# Cleaning up the data

Now we can do some simple data manipulation to make the data more tidy.

In [9]:
# choose only closing prices and rename columns based on the symbols
final_table = final_table[c(1, seq(5, ncol(final_table), 6))]
for (i in 1:length(symbols)) {
  symbol = symbols[[i]]
  column_name = colnames(final_table[i+1])
  final_table = final_table %>%
    rename(!!symbol := column_name)
  i=i+1
}
  
# format Date as date
final_table$Date = as.Date(final_table$Date, "%b %d, %Y")

# sort data with Date
final_table = final_table[order(final_table$Date, decreasing = FALSE),]

# replace "-" with NA, format columns and replace NA's with previous value
final_table = final_table %>% mutate_if(is.character, ~na_if(.,"-"))
final_table = final_table %>% mutate_if(is.character, ~as.numeric(sub(",", "", ., fixed = TRUE)))
final_table = na.locf(final_table)
final_table = distinct(final_table)

Note: Using an external vector in selections is ambiguous.
i Use `all_of(column_name)` instead of `column_name` to silence this message.
i See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.
This message is displayed once per session.


Now let us check whether the table looks fine.

In [10]:
head(final_table, 10)

Date,NFLX,SPOT,TLT
2021-08-31,569.19,234.34,148.83
2021-09-01,582.07,238.38,148.89
2021-09-02,588.55,254.03,149.54
2021-09-03,590.53,249.04,148.18
2021-09-07,606.71,254.72,146.93
2021-09-08,606.05,249.68,147.93
2021-09-09,597.54,246.01,149.72
2021-09-10,598.72,247.76,148.40
2021-09-13,589.29,244.22,149.30
2021-09-14,577.76,243.81,151.11


## Exporting the data into an Excel-file

Finally we can export it into an xlsx-file.

In [11]:
write.xlsx(final_table, file = path, col.names = TRUE, row.names = FALSE)